In [1]:
import nltk

nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd

[nltk_data] Downloading package punkt to C:\Users\Mangarakov
[nltk_data]     Alexandr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Mangarakov
[nltk_data]     Alexandr\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
import os

df = pd.read_csv(os.path.join('../../dataset/raw/train.csv'), names=['label', 'Title', 'Description'])
df['text'] = (df['Title'] + '. ' + df['Description'])
df.drop(columns=['Title', 'Description'], axis=1, inplace=True)
print(df.head())
print(df['text'][1])

   label                                               text
0      3  Wall St. Bears Claw Back Into the Black (Reute...
1      3  Carlyle Looks Toward Commercial Aerospace (Reu...
2      3  Oil and Economy Cloud Stocks' Outlook (Reuters...
3      3  Iraq Halts Oil Exports from Main Southern Pipe...
4      3  Oil prices soar to all-time record, posing new...
Carlyle Looks Toward Commercial Aerospace (Reuters). Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.


In [3]:
df['label']

0         3
1         3
2         3
3         3
4         3
         ..
119995    1
119996    2
119997    2
119998    2
119999    2
Name: label, Length: 120000, dtype: int64

In [4]:
import re

def split_to_sent(text):
    text = re.sub(r"(?<=&lt;).*?(?=&gt;)", " ", text)
    text = re.sub(r"&gt;", " ", text)
    text = re.sub(r"&lt;", " ", text)
    sentences = re.split(
        r"(((?<!\w\.\w.)(?<!\s\w\.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s(?=[A-Z]))|((?<![\,\-\:])\n(?=[A-Z]|\" )))", text)[::4]
    return sentences

In [5]:
def split_to_words(sentence):
    words = re.findall(r"\w+@\w+\.\w+|\+\d{1,3}-\d{3}-\d{3}-\d{2}-\d{2}|\w+", sentence)
    return words

In [6]:
def save_to_file(original, lemmatized, stemmed, id, path):
    with open(os.path.join(path, id), "w") as f:
        for i in range(len(original)):
            if original[i] == "\n":
                print("", file=f)
            else:
                print(original[i], stemmed[i], lemmatized[i], sep="\t", file=f)

In [7]:
from nltk import WordNetLemmatizer
from nltk import SnowballStemmer
import nltk
nltk.download('wordnet')


def process_file(df, path):
    wnl = WordNetLemmatizer()
    sst = SnowballStemmer("english")
    counter = 0
    for index, row in df.iterrows():
        sentences = split_to_sent(row['text'])
        words_dic = []
        counter += 1
        for s in sentences:
            words_dic += split_to_words(s)
            words_dic.append("\n")
        lemmatized = []
        stemmed = []
        original = []
        for w in words_dic:
            w_processed = re.sub(r"[.!?,]$", "", w).lower()
            lemmatized.append(wnl.lemmatize(w_processed))
            stemmed.append(sst.stem(w_processed))
            original.append(w_processed)
        save_to_file(original, lemmatized, stemmed, os.path.join(str(row['label']), f'{str(counter)}.tsv'), path)

[nltk_data] Downloading package wordnet to C:\Users\Mangarakov
[nltk_data]     Alexandr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
df = pd.read_csv(os.path.join('../../dataset/raw/train.csv'), names=['label', 'Title', 'Description'])
df['text'] = (df['Title'] + '. ' + df['Description'])
df.drop(columns=['Title', 'Description'], axis=1, inplace=True)

process_file(df, os.path.join('..', 'assets', 'annotated-corpus', 'train'))

In [9]:
df = pd.read_csv(os.path.join('../../dataset/raw/test.csv'), names=['label', 'Title', 'Description'])
df['text'] = (df['Title'] + '. ' + df['Description'])
df.drop(columns=['Title', 'Description'], axis=1, inplace=True)

process_file(df, os.path.join('..', 'assets', 'annotated-corpus', 'test'))